In [1]:
from sentence_transformers import CrossEncoder, SentenceTransformer, util

/home/zim/.virtualenvs/SI650/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = 'How many people live in Berlin?'
docs = ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.',
        'Germany\'a capital has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.',
        'New York City is famous for the Metropolitan Museum of Art.']

In [3]:
# Cross encoders take two inputs to compare them (e.g., a query an document)

# We get the model from https://huggingface.co/cross-encoder/msmarco-MiniLM-L6-en-de-v1 
# All of these models get specified by a string name, also shown at the top of the page.
# The sentence_tranformers package builds on top of the huggingface library which provides
# a bunch of pretrained models for us to use.
cross_encoder_model = CrossEncoder('cross-encoder/msmarco-MiniLM-L6-en-de-v1', max_length=512)

In [4]:
# Create pairs of inputs for the model
pairs = [(query, doc) for doc in docs]

In [5]:
# A cross-encoder takes a pair (tuple[str,str]) as input
scores = cross_encoder_model.predict(pairs)
scores

array([ 9.022886,  8.658829, -9.72064 ], dtype=float32)

In [ ]:
# A bi-encoder takes one input (str) and turns it into a vector.
# Then, we have to compare those vectors with dot product to estimate
# their relevance (at least in the IR setting)

# This model obtained from https://huggingface.co/sentence-transformers/msmarco-MiniLM-L12-cos-v5
biencoder_model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L12-cos-v5')

In [ ]:
# Encode query and documents. Note that we can encode the list of documents all at once!
# We don't have to encode them one at a time. 
#
# encode() has many useful arguments you should check out if you want to use a bi-encoder in practice
query_emb = biencoder_model.encode(query)
doc_emb = biencoder_model.encode(docs)

In [ ]:
type(doc_emb)

In [ ]:
# See that the document embeddings are a matrix, with one row per document
doc_emb.shape

In [ ]:
# Compute dot score (unnormalized cosine similarity!) between query and all document embeddings.
# These scores are our measure of relevance
#
# NOTE: this dot_score() is going to return a pytorch Tensor object. We'll want to 
# move this to a numpy representation which we can get with cpu() and then we'll
# convert it to a list of values
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

In [ ]:
## Let's actually print the ranking

# Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

# Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

# Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)